In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
import pyarrow.parquet as pq

## Reading data

In [6]:
dir_name = 'C:\Cloud\OneDrive - Emory University\Papers\PASC Diabetes Incidence'
sub_dir_name = '\dom abstract'
outcome_df = pd.read_parquet(dir_name + sub_dir_name + '\pddabs_ipw for cohort membership data.parquet' )


outcome_df.head()

,ID,female,nhwhite,nhblack,hispanic,nhother,age,matchid,index_date,site,...,LOINC_48159_8_gtQ3,LOINC_27297_1_gtQ3,LOINC_50553_7_gtQ3,LOINC_53290_3_gtQ3,LOINC_49524_2_gtQ3,LOINC_29908_1_gtQ3,LOINC_49521_8_gtQ3,LOINC_62423_9_gtQ3,LOINC_24027_5_gtQ3,LOINC_11557_6_gtQ3
0,12MAR202320220187400000003,1.0,1.0,0.0,0.0,0.0,72.0,,2018-08-20,Source5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,12MAR202320220187400000004,0.0,1.0,0.0,0.0,0.0,84.0,,2020-05-21,Source7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12MAR202320220187400000007,1.0,0.0,0.0,0.0,1.0,18.0,,2021-03-08,Source2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,12MAR202320220187400000008,1.0,0.0,1.0,0.0,0.0,60.0,,2022-01-15,Source3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12MAR202320220187400000014,1.0,0.0,0.0,1.0,0.0,47.0,,2022-01-13,Source2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# The below lines were commented out after testing a simple model
# outcome_df = outcome_df.sample(frac=0.1, random_state=1)
# sample_outcome_df.shape
outcome_df.shape

(178109, 2983)

In [8]:
# https://stackoverflow.com/questions/11587782/creating-dummy-variables-in-pandas-for-python
outcome_df = pd.get_dummies(outcome_df,prefix='',prefix_sep='_',
                                           columns=['site','calendar_month','payer_type_primary','payer_type_secondary'],drop_first=True)
outcome_df.shape
# https://www.kdnuggets.com/2020/07/easy-guide-data-preprocessing-python.html
# Also lists one-hot encoding as an option

(178109, 3008)

## Code for Random Forest Classifier
https://scikit-learn.org/stable/modules/grid_search.html

In [9]:
rf = RandomForestClassifier(random_state=1)
# n_estimators = number of trees in the forest
# random_state = controls both the randomness of the bootstrapping of the samples used when building trees (if bootstrap=True), 
# and the sampling of the features to consider when looking for the best split at each node (if max_features < n_features)
min_samples_leaf = [10]
n_estimators = [2000]

# TRIAL 
# min_samples_leaf = [500, 1000]
# n_estimators = [5, 10] # Commented out after testing a simple model

random_grid = {'min_samples_leaf': min_samples_leaf,
               'n_estimators': n_estimators}
print(random_grid)


{'min_samples_leaf': [10, 25], 'n_estimators': [1000, 2000]}


## Train-Test Split

In [10]:
y = outcome_df['COHORT']
X = outcome_df.drop(['COHORT','EXPOSED','UNEXPOSED','HISTORICAL','ID','matchid','index_date'], axis=1)

X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

## Creating different outcome variables from train-test split

In [11]:
y_train_EXPOSED = (y_train == 'exposed').astype(int)
y_train_UNEXPOSED = (y_train == 'unexposed').astype(int)
y_train_HISTORICAL = (y_train == 'historical').astype(int)

y_test_EXPOSED = (y_test == 'exposed').astype(int)
y_test_UNEXPOSED = (y_test == 'unexposed').astype(int)
y_test_HISTORICAL = (y_test == 'historical').astype(int)


In [12]:
import collections
# X_train.nhwhite.value_counts(dropna=False)
collections.Counter(y_train)


Counter({'unexposed': 192566, 'exposed': 60159, 'historical': 60082})

## Grid Search
https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


### Exposed vs Other

In [14]:
gs_rf_EXPOSED = GridSearchCV(rf,
                      param_grid=random_grid,
                      # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring
                      # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
                      scoring = make_scorer(recall_score, average='weighted'),
                      cv=5)
gs_rf_EXPOSED.fit(X_train, y_train_EXPOSED)
gs_rf_EXPOSED.best_params_

{'min_samples_leaf': 10, 'n_estimators': 2000}

In [15]:
gs_rf_EXPOSED.cv_results_

{'mean_fit_time': array([3303.99533954, 5493.84352412, 2092.37502689, 4338.03481636]),
 'std_fit_time': array([ 81.12890016, 601.98655528,  10.48954559, 303.36041632]),
 'mean_score_time': array([28.55400262, 46.8072453 , 18.10110097, 36.50688338]),
 'std_score_time': array([0.92405246, 6.02223454, 0.11429852, 1.56319148]),
 'param_min_samples_leaf': masked_array(data=[10, 10, 25, 25],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 2000, 1000, 2000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 10, 'n_estimators': 1000},
  {'min_samples_leaf': 10, 'n_estimators': 2000},
  {'min_samples_leaf': 25, 'n_estimators': 1000},
  {'min_samples_leaf': 25, 'n_estimators': 2000}],
 'split0_test_score': array([0.82139318, 0.82137719, 0.81597455, 0.81603849]),
 'split1_test_score': array([0.82193664, 0.82195262,

In [16]:
gs_rf_EXPOSED.best_score_

0.8216631980632764

In [17]:
gs_rf_EXPOSED.score(X_test, y_test_EXPOSED)

0.8230096416971433

### Unexposed vs Other

In [18]:
gs_rf_UNEXPOSED = GridSearchCV(rf,
                      param_grid=random_grid,
                      # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring
                      # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
                      scoring = make_scorer(recall_score, average='weighted'),
                      cv=5)
gs_rf_UNEXPOSED.fit(X_train, y_train_UNEXPOSED)
gs_rf_UNEXPOSED.best_params_

{'min_samples_leaf': 10, 'n_estimators': 2000}

In [19]:
gs_rf_UNEXPOSED.cv_results_

{'mean_fit_time': array([2988.59148431, 5168.24266725, 2144.16353421, 3936.7920918 ]),
 'std_fit_time': array([677.09767299, 613.46685969, 198.50112709,  25.48593291]),
 'mean_score_time': array([26.49764538, 46.90005198, 18.89438653, 35.94184675]),
 'std_score_time': array([7.10669089, 5.15039375, 1.12181581, 0.20592627]),
 'param_min_samples_leaf': masked_array(data=[10, 10, 25, 25],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 2000, 1000, 2000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 10, 'n_estimators': 1000},
  {'min_samples_leaf': 10, 'n_estimators': 2000},
  {'min_samples_leaf': 25, 'n_estimators': 1000},
  {'min_samples_leaf': 25, 'n_estimators': 2000}],
 'split0_test_score': array([0.77083533, 0.77089927, 0.75990218, 0.75995013]),
 'split1_test_score': array([0.77676545, 0.7771171 ,

In [20]:
gs_rf_UNEXPOSED.best_score_

0.774365665458136

In [21]:
gs_rf_UNEXPOSED.score(X_test, y_test_UNEXPOSED)

0.7760926830515844

### Historical vs Other

In [22]:
gs_rf_HISTORICAL = GridSearchCV(rf,
                      param_grid=random_grid,
                      # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring
                      # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
                      scoring = make_scorer(recall_score, average='weighted'),
                      cv=5)
gs_rf_HISTORICAL.fit(X_train, y_train_HISTORICAL)
gs_rf_HISTORICAL.best_params_

{'min_samples_leaf': 10, 'n_estimators': 2000}

In [23]:
gs_rf_HISTORICAL.cv_results_

{'mean_fit_time': array([2754.0484858 , 5862.60909104, 2282.45506916, 3835.14636097]),
 'std_fit_time': array([678.17257845, 597.25667339,  37.68531522, 153.7442459 ]),
 'mean_score_time': array([25.93892941, 65.17856493, 21.43345194, 35.86089616]),
 'std_score_time': array([ 2.72580595, 20.67026836,  0.29885496,  0.25414226]),
 'param_min_samples_leaf': masked_array(data=[10, 10, 25, 25],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 2000, 1000, 2000],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 10, 'n_estimators': 1000},
  {'min_samples_leaf': 10, 'n_estimators': 2000},
  {'min_samples_leaf': 25, 'n_estimators': 1000},
  {'min_samples_leaf': 25, 'n_estimators': 2000}],
 'split0_test_score': array([0.88019884, 0.88040664, 0.86792302, 0.86841853]),
 'split1_test_score': array([0.8812538 , 0.88168

In [24]:
gs_rf_HISTORICAL.best_score_

0.8807923080129643

In [29]:
gs_rf_HISTORICAL.score(X_test, y_test_HISTORICAL)

0.8835196030792052

## Final Fit


In [26]:
# https://datatofish.com/numpy-array-to-pandas-dataframe/
y_pred_EXPOSED = gs_rf_EXPOSED.predict(X)
y_pred_proba_EXPOSED = gs_rf_EXPOSED.predict_proba(X)
pd.DataFrame(y_pred_proba_EXPOSED,columns=['other','exposed']).to_csv(dir_name + sub_dir_name + '\pcra201_predicted probability for EXPOSED_min10_ntree2000.csv')
pd.crosstab(y,y_pred_EXPOSED)


col_0,0,1
COHORT,,
exposed,66177,9040
historical,75209,8
unexposed,240296,279


In [27]:

y_pred_UNEXPOSED = gs_rf_UNEXPOSED.predict(X)
y_pred_proba_UNEXPOSED = gs_rf_UNEXPOSED.predict_proba(X)

pd.DataFrame(y_pred_proba_UNEXPOSED,columns=['other','unexposed']).to_csv(dir_name + sub_dir_name + '\pcra201_predicted probability for UNEXPOSED_min10_ntree2000.csv')
pd.crosstab(y,y_pred_UNEXPOSED)



col_0,0,1
COHORT,,
exposed,34677,40540
historical,61129,14088
unexposed,21613,218962


In [28]:
y_pred_HISTORICAL = gs_rf_HISTORICAL.predict(X)
y_pred_proba_HISTORICAL = gs_rf_HISTORICAL.predict_proba(X)
pd.DataFrame(y_pred_proba_HISTORICAL,columns=['other','historical']).to_csv(dir_name + sub_dir_name + '\pcra201_predicted probability for HISTORICAL_min10_ntree2000.csv')
pd.crosstab(y,y_pred_HISTORICAL)

col_0,0,1
COHORT,,
exposed,73734,1483
historical,38716,36501
unexposed,238952,1623
